In [32]:
import os
import json
import pandas as pd
import numpy as np
import tqdm
import scipy.sparse as sp
from matplotlib import pyplot as plt
import seaborn as sns
import mglearn
%matplotlib inline
from IPython.display import display
from pprint import pprint
pd.set_option('display.max_columns', 50)
DATA_PATH='D:\download'
t_e = pd.read_csv(
    os.path.join(DATA_PATH, 'transactions_series_toregress.csv'),
    
)
t_e['DEV0']=t_e['DEV0'].astype(np.int16)
t_e['DEV1']=t_e['DEV1'].astype(np.int16)
t_e['DEV3']=t_e['DEV3'].astype(np.int16)
t_e['DEV4']=t_e['DEV4'].astype(np.int16)
t_e['DEV5']=t_e['DEV5'].astype(np.int16)
t_e['DEV6']=t_e['DEV6'].astype(np.int16)
t_e['feature_5_65']=t_e['feature_5_65'].astype(np.int16)
t_e['feature_5_00']=t_e['feature_5_00'].astype(np.int16)
t_e['feature_5_68']=t_e['feature_5_68'].astype(np.int16)
t_e['feature_5_59']=t_e['feature_5_59'].astype(np.int16)
t_e['feature_5_-1']=t_e['feature_5_-1'].astype(np.int16)
t_e['smallf3']= t_e['smallf3'].astype(np.int16)
t_e['middlef3']=t_e['middlef3'].astype(np.int16)
t_e['bigf3']=t_e['bigf3'].astype(np.int16)
t_e['CM_S']=t_e['CM_S'].astype(np.int16)
t_e['CM_P']=t_e['CM_P'].astype(np.int16)
t_e['CM_R']=t_e['CM_R'].astype(np.int16)
t_e['minus']=t_e['minus'].astype(np.int16)
t_e['feature_5_44']=t_e['feature_5_44'].astype(np.int16)
t_e['feature_3']=t_e['feature_3'].astype(np.int16)
t_e['duration']=t_e['duration'].astype(np.int16)
t_e['element_uid']=t_e['element_uid'].astype(np.int32)
t_e['user_uid']=t_e['user_uid'].astype(np.int32)

t_e=t_e.drop(['Unnamed: 0'],1)
t_e.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1076152 entries, 0 to 1076151
Data columns (total 31 columns):
element_uid       1076152 non-null int32
user_uid          1076152 non-null int32
watched_time_x    1072866 non-null float64
rating            7393 non-null float64
duration          1076152 non-null int16
feature_2         1076152 non-null float64
feature_3         1076152 non-null int16
feature_4         1076152 non-null float64
quantity          1076152 non-null int64
rating_mean       1051603 non-null float64
CM_S              1076152 non-null int16
CM_P              1076152 non-null int16
CM_R              1076152 non-null int16
feature_5_65      1076152 non-null int16
feature_5_00      1076152 non-null int16
feature_5_68      1076152 non-null int16
feature_5_59      1076152 non-null int16
feature_5_-1      1076152 non-null int16
feature_5_44      1076152 non-null int16
smallf3           1076152 non-null int16
middlef3          1076152 non-null int16
bigf3             1

In [33]:
t_e.duration= t_e.duration.map(lambda x: 5 if x == 0 else x)
t_e.watched_time_x=(t_e.watched_time_y/60)/(t_e.duration)
t_e['ts-f1']=t_e['ts-f1'].abs()


In [34]:
t_e[t_e.watched_time_x > 20] = t_e[t_e.watched_time_x > 20].fillna(10.0)
t_e[t_e.watched_time_x > 10] = t_e[t_e.watched_time_x > 10].fillna(8.0)

In [35]:
t_e.watched_time_y=(t_e.watched_time_y/60)
t_e["user_rating_mean"] = t_e.user_uid.map(t_e.groupby("user_uid").rating.mean())
from collections import Counter
coun = pd.DataFrame.from_dict(Counter(t_e['user_uid']), orient='index')
coun['user_uid']=coun.index
coun['quantity_user_views']=coun[0]

coun=coun.reset_index()

coun=coun.drop(['index'], axis=1)
coun=coun.drop([0],1)
t_e=pd.merge(t_e, coun, on='user_uid')
t_e["wt_y_user_mean"] = t_e.user_uid.map(t_e.groupby("user_uid").watched_time_y.mean())
t_e["wt_x_user_mean"] = t_e.user_uid.map(t_e.groupby("user_uid").watched_time_x.mean())
t_e["new_content"]=t_e['ts-f1'].map(lambda x: 1 if x<31 else 0)

In [36]:
t_e=t_e.drop(['minus'],1 )

In [37]:

t_e['quartile__element_views'] = pd.qcut(t_e['quantity'], 4, ['1','2','3','4'])
t_e['quartile__user_views'] = pd.qcut(t_e['quantity_user_views'], 4, ['1','2','3','4'])
t_e['quartile__duration'] = pd.qcut(t_e['duration'], 4, ['1','2','3','4'])
t_e['quartile__wtx'] = pd.qcut(t_e['wt_x_user_mean'], 4, ['1','2','3','4'])


t_e['quartile__f2'] = pd.qcut(t_e['feature_2'], 4, ['1','2','3','4'])
t_e['quartile__f4'] = pd.qcut(t_e['feature_4'], 4, ['1','2','3','4'])


t_e["Q_el_views=1"]=t_e['quartile__element_views'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_el_views=4"]=t_e['quartile__element_views'].map(lambda x: 1 if x=='4' else 0)
t_e["Q_us_views=1"]=t_e['quartile__user_views'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_us_views=4"]=t_e['quartile__user_views'].map(lambda x: 1 if x=='4' else 0)
t_e["Q_duration=1"]=t_e['quartile__duration'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_duration=4"]=t_e['quartile__duration'].map(lambda x: 1 if x=='4' else 0)
t_e["Q_wtx=1"]=t_e['quartile__wtx'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_wtx=4"]=t_e['quartile__wtx'].map(lambda x: 1 if x=='4' else 0)


t_e["Q_f2=1"]=t_e['quartile__f2'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_f2=4"]=t_e['quartile__f2'].map(lambda x: 1 if x=='4' else 0)
t_e["Q_f4=1"]=t_e['quartile__f4'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_f4=4"]=t_e['quartile__f4'].map(lambda x: 1 if x=='4' else 0)
t_e=t_e.drop(['quartile__element_views', 'quartile__user_views',  'quartile__duration', 'quartile__wtx', 'quartile__f2', 'quartile__f4'],1)
t_e['new_content']=t_e['new_content'].astype(np.int16)
t_e['Q_el_views=1']=t_e['Q_el_views=1'].astype(np.int16)
t_e['Q_el_views=4']=t_e['Q_el_views=4'].astype(np.int16)
t_e['Q_us_views=1']=t_e['Q_us_views=1'].astype(np.int16)
t_e['Q_us_views=4']=t_e['Q_us_views=4'].astype(np.int16)
t_e['Q_duration=1']=t_e['Q_duration=1'].astype(np.int16)
t_e['Q_duration=4']=t_e['Q_duration=4'].astype(np.int16)
t_e['Q_wtx=1']=t_e['Q_wtx=1'].astype(np.int16)
t_e['Q_wtx=4']=t_e['Q_wtx=4'].astype(np.int16)
t_e['Q_f2=1']=t_e['Q_f2=1'].astype(np.int16)
t_e['Q_f2=4']=t_e['Q_f2=1'].astype(np.int16)
t_e['Q_f4=1']=t_e['Q_f4=1'].astype(np.int16)
t_e['Q_f4=4']=t_e['Q_f4=1'].astype(np.int16)
t_e.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1076152 entries, 0 to 1076151
Data columns (total 47 columns):
element_uid            1076152 non-null int32
user_uid               1076152 non-null int32
watched_time_x         1076152 non-null float64
rating                 245075 non-null float64
duration               1076152 non-null int64
feature_2              1076152 non-null float64
feature_3              1076152 non-null int16
feature_4              1076152 non-null float64
quantity               1076152 non-null int64
rating_mean            1053930 non-null float64
CM_S                   1076152 non-null int16
CM_P                   1076152 non-null int16
CM_R                   1076152 non-null int16
feature_5_65           1076152 non-null int16
feature_5_00           1076152 non-null int16
feature_5_68           1076152 non-null int16
feature_5_59           1076152 non-null int16
feature_5_-1           1076152 non-null int16
feature_5_44           1076152 non-null int16
smal

In [38]:
t_e.isnull().sum()/t_e.shape[0]

element_uid            0.000000
user_uid               0.000000
watched_time_x         0.000000
rating                 0.772267
duration               0.000000
feature_2              0.000000
feature_3              0.000000
feature_4              0.000000
quantity               0.000000
rating_mean            0.020649
CM_S                   0.000000
CM_P                   0.000000
CM_R                   0.000000
feature_5_65           0.000000
feature_5_00           0.000000
feature_5_68           0.000000
feature_5_59           0.000000
feature_5_-1           0.000000
feature_5_44           0.000000
smallf3                0.000000
middlef3               0.000000
bigf3                  0.000000
ts-f1                  0.000000
watched_time_y         0.000000
DEV0                   0.000000
DEV1                   0.000000
DEV3                   0.000000
DEV4                   0.000000
DEV5                   0.000000
DEV6                   0.000000
user_rating_mean       0.308817
quantity

In [39]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
t_e.user_rating_mean=t_e.user_rating_mean.fillna(5.0) 
t_e=t_e.drop(['user_rating_mean'], 1)
transactions_exp=t_e


transactions_exp.rating_mean=transactions_exp.rating_mean.fillna(5.0) 

transactions_exp['worst']=transactions_exp.watched_time_x.map(lambda x:1 if (x < 0.2 and x >0.005) else 0)
transactions_exp[transactions_exp.worst == 1]=transactions_exp[transactions_exp.worst == 1].fillna(2.0)



X_train = transactions_exp[~transactions_exp.rating.isnull()]
X_train=X_train.drop(['rating'],1)
X_test = transactions_exp[transactions_exp.rating.isnull()]
X_test=X_test.drop(['rating'],1)
y_train = transactions_exp[~transactions_exp.rating.isnull()].rating.astype(np.float)
y_test = transactions_exp[transactions_exp.rating.isnull()].rating


model2 = RandomForestRegressor( n_estimators=522, max_depth=8, random_state=241, n_jobs=-1)

model2.fit(X_train, y_train)
submit1 = pd.DataFrame(model2.predict(X_test))
#kf = KFold(n_splits=5, shuffle=True, random_state=241) 

estimator=xgb.XGBRegressor(learning_rate=0.1, max_depth=8, n_estimators=700, n_jobs=-1  )

    
estimator.fit(X_train, y_train)
submit2 = pd.DataFrame(estimator.predict(X_test))

#model2 = RandomForestRegressor( n_estimators=522, max_depth=8, random_state=241, n_jobs=-1)
#score=cross_val_score(model2, X_train, y_train, cv=kf, n_jobs=-1, scoring='mean_squared_error')
#print("Score is", score.mean())


In [9]:
model2.fit(X_train, y_train)
submit1 = pd.DataFrame(model2.predict(X_test))

In [11]:

estimator=xgb.XGBRegressor(learning_rate=0.1, max_depth=8, n_estimators=700, n_jobs=-1  )

    
estimator.fit(X_train, y_train)
submit2 = pd.DataFrame(estimator.predict(X_test))


In [14]:
importances = estimator.feature_importances_
indices = np.argsort(importances)[::-1]
feature_names=X_train.columns
print("Feature importances:")
for f, idx in enumerate(indices):
    print("{:2d}. feature '{:5}' ({:.4f})".format(f+1, feature_names[idx], importances[idx]))


Feature importances:
 1. feature 'rating' (0.1324)
 2. feature 'element_uid' (0.1179)
 3. feature 'user_uid' (0.1141)


IndexError: index 29 is out of bounds for axis 0 with size 3

In [ ]:
Score is -0.09979454312919549 - без кучи признаков
Score is -0.05015586917549114 - с фил га 6.0 юзер рейтинг мин
нужно попробовать без филл на()

In [40]:
X_train = transactions_exp[~transactions_exp.rating.isnull()]
#X_train.rating_mean=X_train.rating_mean.fillna(6.0)
X_train=X_train.drop([
'watched_time_x',
'duration',
'feature_2',
'feature_3',
'feature_4',
'quantity',
'rating_mean',
'CM_S',
'CM_P',
'CM_R',
'feature_5_65',
'feature_5_00',
'feature_5_68',
'feature_5_59',
'feature_5_-1',
'feature_5_44',
'smallf3',
'middlef3',
'bigf3',
'ts-f1',
'watched_time_y',
'DEV0',
'DEV1',
'DEV3',
'DEV4',
'DEV5',
'DEV6',
'quantity_user_views',
'wt_y_user_mean',         
'wt_x_user_mean',
'new_content',
'Q_el_views=1',
'Q_el_views=4',
'Q_duration=1',
#'user_rating_mean',
'Q_duration=4',
'Q_wtx=1',
'Q_wtx=4',
'Q_f2=1',
'Q_f2=4',
'Q_f4=1',
'Q_f4=4',
'Q_us_views=4',
'Q_us_views=1'],1)
X_test=X_test.drop([
'watched_time_x',
'duration',
'feature_2',
'feature_3',
'feature_4',
'quantity',
'rating_mean',
'CM_S',
'CM_P',
'CM_R',
'feature_5_65',
'feature_5_00',
'feature_5_68',
'feature_5_59',
'feature_5_-1',
'feature_5_44',
'smallf3',
'middlef3',
'bigf3',
'ts-f1',
'watched_time_y',
'DEV0',
'DEV1',
'DEV3',
'DEV4',
'DEV5',
'DEV6',
'quantity_user_views',
'wt_y_user_mean',         
'wt_x_user_mean',
'new_content',
'Q_el_views=1',
'Q_el_views=4',
'Q_duration=1',
#'user_rating_mean',
'Q_duration=4',
'Q_wtx=1',
'Q_wtx=4',
'Q_f2=1',
'Q_f2=4',
'Q_f4=1',
'Q_f4=4',
'Q_us_views=4',
'Q_us_views=1'],1)
#submit[0]=submit[0].round()
#submit[0]=submit[0].astype(np.int16)
X_test=X_test.reset_index()
X_test=X_test.drop(['index'], 1)

OLD=pd.concat([X_test, submit2], 1)
#OLD.rating=OLD.rating.round()
OLD['rating']=OLD[0]
OLD=OLD.drop([0],1)
#ratings=ratings.drop(['ts'],1)

OLD=OLD.append(X_train)
OLD.to_csv("SeriesRatingXGB4.csv")
OLD=pd.concat([X_test, submit1], 1)
#OLD.rating=OLD.rating.round()
OLD['rating']=OLD[0]
OLD=OLD.drop([0],1)

OLD=OLD.append(X_train)
OLD.to_csv("SeriesRatingRF4.csv")

In [18]:
OLD=pd.concat([X_test, submit1], 1)
#OLD.rating=OLD.rating.round()
OLD['rating']=OLD[0]
OLD=OLD.drop([0],1)
OLD=OLD.append(X_train)
OLD.to_csv("SeriesRatingRF.csv")

In [79]:
OLD.to_csv("SeriesRating(withoutstart).csv")

In [20]:
X_train.info()
OLD=OLD.append(X_train)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 427890 entries, 0 to 427889
Data columns (total 5 columns):
element_uid     427890 non-null int32
user_uid        427890 non-null int32
rating          427890 non-null float64
Q_us_views=1    427890 non-null int16
Q_us_views=4    427890 non-null int16
dtypes: float64(1), int16(2), int32(2)
memory usage: 8.2 MB


In [21]:
OLD=OLD.drop(['Q_us_views=4', 'Q_us_views=1'], 1)
OLD.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1076152 entries, 0 to 427889
Data columns (total 3 columns):
element_uid    1076152 non-null int32
rating         1076152 non-null float64
user_uid       1076152 non-null int32
dtypes: float64(1), int32(2)
memory usage: 24.6 MB


In [22]:
#OLD=OLD.drop(['Unnamed: 0'], 1)
OLD.to_csv("SeriesRatingRF.csv")

In [23]:
OLD.rating

0         10.000000
1         10.000000
2         10.000000
3         10.000000
4         10.000000
5         10.000000
6          5.442965
7          5.025445
8          5.688655
9          6.097752
10         5.015592
11         3.509259
12         5.128670
13         5.193659
14        10.000000
15         9.660503
16         5.491107
17         5.278297
18         7.497592
19         6.300915
20         7.513081
21        10.000000
22         9.997987
23        10.000000
24        10.000000
25        10.000000
26        10.000000
27        10.000000
28        10.000000
29         8.325056
            ...    
427860    10.000000
427861    10.000000
427862    10.000000
427863     2.000000
427864     2.000000
427865     2.000000
427866     2.000000
427867     2.000000
427868     2.000000
427869     2.000000
427870     2.000000
427871     8.000000
427872     8.000000
427873     8.000000
427874     8.000000
427875     2.000000
427876     8.000000
427877     2.000000
427878     2.000000


In [14]:
#OLD=OLD.drop(['Unnamed: 0'], 1)
OLD.to_csv("SeriesRating.csv")

In [2]:
e=pd.read_csv('SeriesRatingXGB.csv')


FileNotFoundError: File b'SeriesRatingXGB.csv' does not exist

In [ ]:
model2.fit(X_train, y_train)
submit1 = pd.DataFrame(estimator.predict(X_test))
submit1.to_csv("sample_submissionrf02.03.csv")


estimator=xgb.XGBRegressor(learning_rate=0.1, max_depth=8, n_estimators=700, n_jobs=-1  )
score1=cross_val_score(estimator, X_train, y_train, cv=kf, n_jobs=-1, scoring='mean_squared_error')
print("Score is", score1.mean())
estimator.fit(X_train, y_train)
submit = pd.DataFrame(estimator.predict(X_test))
submit.to_csv("sample_submissionxgb02.03.csv")